# YOLO Model Training Pipeline

This notebook executes the complete training workflow for YOLOv8n object detection model using Pascal VOC 2012 dataset. Trains on real-world dataset with configuration defined in notebook 02, tracks metrics with MLflow, and validates trained model.

## Training Workflow:
1. Load model configuration from notebook 02
2. Initialize YOLO model with pretrained COCO weights
3. Execute training on Pascal VOC 2012 dataset (3000-5000 filtered images)
4. Track training progress and metrics with MLflow
5. **Register model in MLflow Model Registry** ✨
6. Validate trained model on validation dataset
7. Verify best.pt checkpoint ready for prediction phase

## Training Duration:
- GPU: 30-60 minutes
- CPU: 3-4 hours

## Dataset: 
Pascal VOC 2012 (~70% train, 15% val, 15% test split)

## MLflow Integration:
- **Experiments tab:** Métricas, parámetros, artifacts
- **Models tab:** Versiones registradas con staging/producción 🆕


##  IMPORTANTE: Ruta Única del Modelo

Para evitar confusiones, este notebook usa **UNA SOLA RUTA** para el modelo entrenado:

```
models/best.pt
```

**Esta es la ÚNICA ubicación que importa.** Todas las demás rutas son temporales:
- `runs/detect/yolo_run/weights/best.pt` → Ruta temporal de YOLO (se copia a models/)
- MLflow artifacts → Copia automática para tracking

**Para usar el modelo en predicciones/API:**
```python
from ultralytics import YOLO
model = YOLO('models/best.pt')  # ← SIEMPRE usa esta ruta
```

**NO uses rutas de `runs/` porque se borran al re-entrenar.**

In [12]:
import yaml
from pathlib import Path
import mlflow
import os
import torch
import shutil
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image
import sys

# Project structure
PROJECT_ROOT = Path('../').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
MODELS_DIR = PROJECT_ROOT / 'models'
RUNS_DIR = PROJECT_ROOT / 'runs'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Agregar carpeta app al path para importar mlflow_utils
sys.path.insert(0, str(PROJECT_ROOT / 'app'))
from mlflow_utils import setup_mlflow, MLflowYOLOTracker

# Reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Configurar MLflow usando función helper
tracker = setup_mlflow(PROJECT_ROOT)

print("\nYOLO MODEL CONFIGURATION")
print("=" * 60)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Dir: {DATA_DIR}")
print(f"Models Dir: {MODELS_DIR}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("=" * 60)

MLflow Configuration
Tracking URI: file:///C:/Users/jordy/OneDrive/Desktop/iaaaa/iajordy2/runs/mlflow
Experiment: yolo_3class_detection

YOLO MODEL CONFIGURATION
Project Root: C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2
Data Dir: C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\data
Models Dir: C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\models
CUDA Available: True
GPU: NVIDIA GeForce RTX 3080 Laptop GPU


Stage 1: Environment Initialization and MLflow Setup

This stage sets up the training environment with reproducibility settings and experiment tracking configuration.

Components:
- PyTorch and NumPy: Deep learning framework and numerical operations
- MLflow: Experiment tracking and model versioning
- YOLO: Ultralytics YOLOv8 object detection framework

Reproducibility:
- Fixed seeds (42) for NumPy, PyTorch CPU, and CUDA GPU
- Ensures identical results across different runs and machines

MLflow configuration:
- Sets tracking URI to local mlruns directory
- Creates experiment named "yolo_3class_detection"
- All training metrics will be logged and retrievable

In [13]:
# Configuration (from notebook 02)
MODEL_NAME = 'yolov8n'
PRETRAINED_WEIGHTS = 'yolov8n.pt'
NUM_CLASSES = 3
CLASS_NAMES = ['person', 'car', 'dog']

TRAINING_CONFIG = {
    'epochs': 50,
    'batch_size': 16,
    'imgsz': 416,
    'patience': 10,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'seed': SEED,
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
}

print("\n[1] Loading Configuration")
print("-" * 60)
print(f"Model: {MODEL_NAME}")
print(f"Classes: {NUM_CLASSES} ({', '.join(CLASS_NAMES)})")
print(f"\nTraining config:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")


[1] Loading Configuration
------------------------------------------------------------
Model: yolov8n
Classes: 3 (person, car, dog)

Training config:
  epochs: 50
  batch_size: 16
  imgsz: 416
  patience: 10
  device: 0
  seed: 42
  lr0: 0.01
  lrf: 0.01
  momentum: 0.937
  weight_decay: 0.0005
  warmup_epochs: 3.0
  warmup_momentum: 0.8


Stage 2: Load Configuration from Notebook 02

This stage replicates the model and training configuration defined in notebook 02.

Configuration includes:
- Model name and pretrained weights
- Dataset specification (3 classes)
- All training hyperparameters

This duplication ensures that notebook 03 is self-contained and can be executed independently after notebook 02.

In [ ]:

from mlflow.tracking import MlflowClient
import mlflow.pyfunc

# Wrapper personalizado para modelo YOLO en MLflow
class YOLOWrapper(mlflow.pyfunc.PythonModel):
    """Wrapper para hacer modelos YOLO compatibles con MLflow Model Registry"""
    
    def load_context(self, context):
        """Carga el modelo YOLO desde artifacts"""
        from ultralytics import YOLO
        model_path = context.artifacts["model"]
        self.model = YOLO(model_path)
    
    def predict(self, context, model_input):
        """Ejecuta predicción con el modelo YOLO"""
        return self.model.predict(model_input)

# Iniciar run de MLflow con tags descriptivos
tracker.start_run(
    run_name='yolo_training_initial',
    tags={
        'model_type': MODEL_NAME,
        'dataset': 'pascal_voc_2012',
        'training_phase': 'initial',
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu'
    }
)

# Loguear parámetros de forma estructurada
tracker.log_training_params(
    model_name=MODEL_NAME,
    num_classes=NUM_CLASSES,
    class_names=CLASS_NAMES,
    config=TRAINING_CONFIG,
    dataset_info={
        'source': 'Pascal VOC 2012',
        'classes': NUM_CLASSES,
        'train_split': '~70%',
        'val_split': '~15%',
        'test_split': '~15%'
    },
)

print("\n[2] Starting Training")
print("-" * 60)

# Inicializar modelo
model = YOLO(PRETRAINED_WEIGHTS)

try:
    results = model.train(
        data=str(DATA_DIR / 'data.yaml'),
        epochs=TRAINING_CONFIG['epochs'],
        imgsz=TRAINING_CONFIG['imgsz'],
        batch=TRAINING_CONFIG['batch_size'],
        device=TRAINING_CONFIG['device'],
        patience=TRAINING_CONFIG['patience'],
        seed=TRAINING_CONFIG['seed'],
        save=True,
        exist_ok=True,
        name='yolo_run',
        verbose=True
    )
    
    print("\n✓ Training completed successfully")
    
    # RUTA ÚNICA DEL MODELO (sin confusiones)
    yolo_run_dir = RUNS_DIR / 'detect' / 'yolo_run'
    best_model_path = yolo_run_dir / 'weights' / 'best.pt'
    
    if best_model_path.exists():
        # 1. Loguear métricas finales
        if results:
            metrics = tracker.log_metrics_from_yolo(results)
            print(f"\n✓ Final metrics logged:")
            for key, value in metrics.items():
                print(f"  {key}: {value:.4f}")
        
        # 2. Loguear gráficas y configs como artifacts
        plots_to_log = ['results.png', 'confusion_matrix.png', 'F1_curve.png']
        for plot_file in plots_to_log:
            plot_path = yolo_run_dir / plot_file
            if plot_path.exists():
                mlflow.log_artifact(str(plot_path), artifact_path='plots')
        
        args_yaml = yolo_run_dir / 'args.yaml'
        if args_yaml.exists():
            mlflow.log_artifact(str(args_yaml), artifact_path='config')
        
        # 3. Loguear modelo como artifact simple (para backup)
        mlflow.log_artifact(str(best_model_path), artifact_path='model')
        
        # 4. COPIAR modelo a MODELS_DIR (RUTA ÚNICA para uso posterior)
        model_final_path = MODELS_DIR / 'best.pt'
        shutil.copy2(str(best_model_path), str(model_final_path))
        print(f"✓ Model copied to: {model_final_path}")
        
        # 5. REGISTRAR MODELO EN MODEL REGISTRY (formato MLflow)
        print("\n💾 Registering model in MLflow Model Registry...")
        
        # Crear conda environment para reproducibilidad
        conda_env = {
            'channels': ['defaults'],
            'dependencies': [
                f'python={sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}',
                'pip',
                {
                    'pip': [
                        f'ultralytics=={__import__("ultralytics").__version__}',
                        f'torch=={torch.__version__}',
                        f'numpy=={np.__version__}',
                    ]
                }
            ],
            'name': 'yolo_env'
        }
        
        # Guardar modelo en formato MLflow con wrapper
        artifacts = {"model": str(best_model_path)}
        
        mlflow.pyfunc.log_model(
            artifact_path="yolo_model",
            python_model=YOLOWrapper(),
            artifacts=artifacts,
            conda_env=conda_env,
            registered_model_name='yolo_3class_detector'
        )
        
        print("✓ Model registered in Model Registry: yolo_3class_detector")
        
        # 6. Loguear tags y métricas de éxito
        mlflow.set_tag('model_path', str(model_final_path))
        mlflow.log_metric('training_success', 1)
        tracker.end_run(status='FINISHED')
        
        print(f"\n" + "="*60)
        print("TRAINING COMPLETED SUCCESSFULLY")
        print("="*60)
        print(f"Model path (ÚNICA): {model_final_path}")
        print(f"MLflow run ID: {tracker.run_id}")
        print(f"Registered model: yolo_3class_detector")
        print(f"View in MLflow UI: http://localhost:5001")
        print(f"  → Experiments tab: Ver métricas y artifacts")
        print(f"  → Models tab: Ver versiones registradas")
        print("="*60)
        print(f"\n IMPORTANTE: Usa SIEMPRE esta ruta para el modelo:")
        print(f"   {model_final_path}")
        print("="*60)
        
    else:
        mlflow.log_metric('training_success', 0)
        tracker.end_run(status='FAILED')
        print(f"\n✗ Training failed - best.pt not found at {best_model_path}")
        
except Exception as e:
    print(f"\n✗ Training failed with error: {e}")
    mlflow.log_metric('training_success', 0)
    mlflow.log_param('error_message', str(e))
    tracker.end_run(status='FAILED')
    raise



c:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\.venv\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


✓ Logged 20 parameters to MLflow

[2] Starting Training
------------------------------------------------------------
New https://pypi.org/project/ultralytics/8.4.12 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.10  Python-3.10.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\data\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, li

2026/02/05 18:28:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



✓ Training completed successfully
✓ Logged 7 metrics to MLflow

✓ Final metrics logged:
  mAP50: 0.7562
  mAP50_95: 0.5244
  precision: 0.8084
  recall: 0.6745
  mAP50_class_0: 0.5101
  mAP50_class_1: 0.5574
  mAP50_class_2: 0.5058
✓ Model copied to: C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\models\best.pt

💾 Registering model in MLflow Model Registry...


c:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\.venv\lib\site-packages\mlflow\tracing\provider.py:757: FutureWarning: Passing a Python object as `python_model` causes it to be serialized using CloudPickle, it requires exercising caution as Python object serialization mechanisms may execute arbitrary code during deserialization.Consider using a file path (str or Path) instead. See https://mlflow.org/docs/latest/ml/model/models-from-code/ for details.
  result = f(*args, **kwargs)


Registered model 'yolo_3class_detector' already exists. Creating a new version of this model...
Created version '1' of model 'yolo_3class_detector'.


✓ Model registered in Model Registry: yolo_3class_detector
✓ MLflow run ended with status: FINISHED

TRAINING COMPLETED SUCCESSFULLY
Model path (ÚNICA): C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\models\best.pt
MLflow run ID: 5fabef836ad541baad59d462b10e78e2
Registered model: yolo_3class_detector
View in MLflow UI: http://localhost:5001
  → Experiments tab: Ver métricas y artifacts
  → Models tab: Ver versiones registradas

💡 IMPORTANTE: Usa SIEMPRE esta ruta para el modelo:
   C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\models\best.pt


Stage 3: Execute Training with Professional MLflow Integration

This stage implements production-grade MLflow tracking using modular utilities.

## Key Improvements:

### 1. **Structured Logging** 
- Uses `MLflowYOLOTracker` class for consistent logging
- Separates params, metrics, and artifacts clearly
- Adds descriptive tags for easy experiment filtering

### 2. **Artifact Management**
- Automatically logs training plots (results.png, confusion_matrix.png, etc.)
- Logs model weights with proper organization
- Logs training configuration for reproducibility

### 3. **Model Versioning** 
- Semantic versioning (v1.0.0, v1.1.0, etc.)
- Metadata file with training details
- Differentiates between training and retraining

### 4. **Error Handling** 
- Try-catch for robust error capture
- Status logging (FINISHED/FAILED)
- Error messages logged to MLflow

### 5. **No YOLO MLflow Conflict** 
- **CRITICAL FIX**: Removed `mlflow=False` parameter
- YOLO Ultralytics doesn't have built-in MLflow support
- We handle ALL logging manually with our tracker

## What Gets Logged:

**Parameters:**
- Model configuration (epochs, batch_size, lr, etc.)
- Dataset information (source, splits, classes)
- System info (GPU, device)

**Metrics:**
- mAP50, mAP50-95 (mean Average Precision)
- Precision, Recall (per class and overall)
- Class-specific metrics

**Artifacts:**
- best.pt model file
- Training plots (loss curves, metrics, confusion matrix)
- Configuration YAML
- Model version metadata JSON

**Tags:**
- model_type, dataset, training_phase
- gpu, timestamp
- Custom tags for filtering

## Benefits:

 **Reproducibility**: All parameters and configs logged  
 **Comparison**: Easy to compare runs in MLflow UI  
 **Debugging**: Plots and logs available for analysis  
 **Deployment**: Model artifacts ready for production  
 **Audit Trail**: Complete history of experiments  

View results at: http://localhost:5001

In [15]:

# RUTA ÚNICA DEL MODELO ENTRENADO
best_model_path = MODELS_DIR / 'best.pt'

if best_model_path.exists():
    # Iniciar run de validación
    tracker.start_run(
        run_name='yolo_validation_initial',
        tags={
            'model_type': MODEL_NAME,
            'validation_phase': 'post_training',
            'model_checkpoint': 'best.pt'
        }
    )
    
    # Loguear parámetros de validación
    tracker.log_training_params(
        model_name=MODEL_NAME,
        num_classes=NUM_CLASSES,
        class_names=CLASS_NAMES,
        config={
            'imgsz': TRAINING_CONFIG['imgsz'],
            'batch': TRAINING_CONFIG['batch_size'],
            'device': TRAINING_CONFIG['device']
        },
        dataset_info={
            'split': 'validation',
            'source': 'Pascal VOC 2012'
        }
    )
    
    print("\n[3] Running Validation")
    print("-" * 60)
    print(f"Loading model from: {best_model_path}")
    
    # Cargar y validar modelo
    best_model = YOLO(str(best_model_path))
    val_results = best_model.val(
        data=str(DATA_DIR / 'data.yaml'),
        imgsz=TRAINING_CONFIG['imgsz'],
        batch=TRAINING_CONFIG['batch_size'],
        device=TRAINING_CONFIG['device'],
        verbose=False
    )
    
    # Loguear métricas usando tracker
    metrics = tracker.log_metrics_from_yolo(val_results)
    
    if metrics:
        print("\n✓ Validation Metrics:")
        for metric_name, metric_value in metrics.items():
            print(f"  {metric_name}: {metric_value:.4f}")
    
    tracker.end_run(status='FINISHED')
    print("\n✓ Validation completed and logged to MLflow")
    
else:
    print(f"\n✗ Cannot validate: Model not found at {best_model_path}")
    print("   Run the training cell first!")


✓ Logged 8 parameters to MLflow

[3] Running Validation
------------------------------------------------------------
Loading model from: C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\models\best.pt
Ultralytics 8.4.10  Python-3.10.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192MiB)
Model summary (fused): 73 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 747.1128.9 MB/s, size: 83.5 KB)
val: Scanning C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\data\labels\val.cache... 434 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 434/434  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 28/28 10.3it/s 2.7s0.1s
                   all        434       1040      0.861      0.646      0.759      0.527
Speed: 0.6ms preprocess, 1.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\runs\detect\v

Stage 4: Model Validation and Metrics Logging

This stage validates the trained model on the validation dataset and logs performance metrics to MLflow.

Validation process:
1. Load best.pt model checkpoint
2. Execute model.val() on validation dataset with same hyperparameters used in training
3. Extract performance metrics from validation results

Metrics computed:
- mAP50: Mean Average Precision at IoU threshold 0.50
- mAP50_95: Mean Average Precision averaged over IoU thresholds 0.50-0.95
- precision: Proportion of detections that are correct
- recall: Proportion of ground truth objects that are detected

MLflow logs:
- All metrics for experiment tracking and comparison
- Model checkpoint path reference
- Validation dataset specification

## Bonus: Comparing Experiments with MLflow

The new MLflow integration allows easy comparison of training runs.
This helps select the best model for production deployment.

In [17]:
# ============================================================================
# COMPARE MULTIPLE TRAINING RUNS (OPTIONAL)
# ============================================================================

# Ejemplo: Comparar los últimos 3 runs
import mlflow
import pandas as pd

experiment = mlflow.get_experiment_by_name('yolo_3class_detection')
if experiment:
    # Obtener últimos runs
    runs = mlflow.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["start_time DESC"],
        max_results=3
    )
    
    if len(runs) > 0:
        print("\n COMPARISON OF RECENT RUNS")
        print("=" * 80)
        
        # Seleccionar columnas relevantes
        comparison_cols = [
            'run_id', 
            'tags.mlflow.runName',
            'metrics.mAP50',
            'metrics.mAP50_95',
            'metrics.precision',
            'metrics.recall',
            'params.train_epochs',
            'params.model_version'
        ]
        
        # Filtrar columnas existentes
        available_cols = [col for col in comparison_cols if col in runs.columns]
        comparison_df = runs[available_cols]
        
        # Renombrar para claridad
        comparison_df.columns = [col.split('.')[-1] for col in comparison_df.columns]
        
        print(comparison_df.to_string(index=False))
        print("\n" + "=" * 80)
        print(" TIP: View detailed comparison in MLflow UI at http://localhost:5001")
        print("   Select runs → Compare → View metrics side-by-side")
    else:
        print("No runs found yet. Train the model first!")
else:
    print("Experiment not found. Train the model first!")



 COMPARISON OF RECENT RUNS
                          run_id               runName train_epochs
5fabef836ad541baad59d462b10e78e2 yolo_training_initial           50
f5a4eb4955db476895ca57205ac641a8 yolo_training_initial           50

 TIP: View detailed comparison in MLflow UI at http://localhost:5001
   Select runs → Compare → View metrics side-by-side


In [19]:
# ============================================================================
# VERIFICAR MODELO EN MLFLOW MODEL REGISTRY
# ============================================================================
import mlflow
from mlflow.tracking import MlflowClient
from pathlib import Path

client = MlflowClient()
model_name = 'yolo_3class_detector'

print("\n VERIFICACIÓN DEL MODELO")
print("=" * 80)

# 1. Verificar archivo local
PROJECT_ROOT = Path('../').resolve()
local_model_path = PROJECT_ROOT / 'models' / 'best.pt'

if local_model_path.exists():
    size_mb = local_model_path.stat().st_size / (1024 * 1024)
    print(f"\n Modelo local encontrado:")
    print(f"    Ruta: {local_model_path}")
    print(f"    Tamaño: {size_mb:.2f} MB")
else:
    print(f"\n Modelo local NO encontrado en: {local_model_path}")

# 2. Verificar Model Registry
try:
    registered_model = client.get_registered_model(model_name)
    versions = client.search_model_versions(f"name='{model_name}'")
    
    print(f"\n Modelo en Model Registry:")
    print(f"     Nombre: {model_name}")
    print(f"    Versiones: {len(versions)}")
    
    if versions:
        latest_version = versions[0]
        print(f"\n    Última versión:")
        print(f"      Versión: {latest_version.version}")
        print(f"      Stage: {latest_version.current_stage}")
        print(f"      Run ID: {latest_version.run_id}")
        print(f"      Creada: {latest_version.creation_timestamp}")
        
except Exception as e:
    print(f"\n jecuta la celda de entrenamiento primero")
    print(f"   Error: {e}")

# 3. Instrucciones de uso
print(f"\n CÓMO USAR EL MODELO:")
print(f"   " + "="*76)
print(f"   # Opción 1: Desde archivo local (RECOMENDADO)")
print(f"   from ultralytics import YOLO")
print(f"   model = YOLO('models/best.pt')")
print(f"   results = model.predict('imagen.jpg')")
print(f"   ")
print(f"   # Opción 2: Desde MLflow Model Registry")
print(f"   import mlflow")
print(f"   model_uri = 'models:/{model_name}/latest'")
print(f"   loaded_model = mlflow.pyfunc.load_model(model_uri)")
print(f"   " + "="*76)

print(f"\n Ver en MLflow UI: http://localhost:5001")
print(f"   → Pestaña 'Models' → '{model_name}'")
print(f"   → Pestaña 'Experiments' → 'yolo_3class_detection'")

print("\n" + "=" * 80)



 VERIFICACIÓN DEL MODELO

 Modelo local encontrado:
    Ruta: C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\models\best.pt
    Tamaño: 5.92 MB

 Modelo en Model Registry:
     Nombre: yolo_3class_detector
    Versiones: 1

    Última versión:
      Versión: 1
      Stage: None
      Run ID: 1504bddfeae34f428ce72e7a605d5486
      Creada: 1770334106825

 CÓMO USAR EL MODELO:
   # Opción 1: Desde archivo local (RECOMENDADO)
   from ultralytics import YOLO
   model = YOLO('models/best.pt')
   results = model.predict('imagen.jpg')
   
   # Opción 2: Desde MLflow Model Registry
   import mlflow
   model_uri = 'models:/yolo_3class_detector/latest'
   loaded_model = mlflow.pyfunc.load_model(model_uri)

 Ver en MLflow UI: http://localhost:5001
   → Pestaña 'Models' → 'yolo_3class_detector'
   → Pestaña 'Experiments' → 'yolo_3class_detection'



##  Verificar Modelo en MLflow

Ejecuta la celda siguiente para verificar que el modelo está correctamente guardado en:

1. ** Local:** `models/best.pt` 
2. ** MLflow Artifacts:** Como artifact del run
3. ** Model Registry:** En la pestaña "Models" de MLflow UI

El modelo ahora aparecerá en **"Models" → "yolo_3class_detector"** con versionado automático.
